# Airbnb Data Project Round 2
So upon deep reflection, I have decided to forsake Kaggle and to search for data sources on the internet, so that I go through the process of looking for data. After a few days of searching, I chanced upon this website [Inside Airbnb](http://insideairbnb.com/get-the-data.html) that had a trove of airbnb data from a anti-airbnb advocate. I have downloaded the Nov 2015 SF data and it looks promising so far, so let's see what we can do with it.

In [6]:
import pandas as pd

# Data sourced from Inside Airbnb (http://insideairbnb.com/get-the-data.html)
# A million thanks to Murray Cox
calendar = pd.read_csv("calendar.csv")
calendar2 = pd.read_csv("calendar2.csv")
listings = pd.read_csv("listings.csv")
listings2 = pd.read_csv("listings2.csv")
nhoods = pd.read_csv("neighbourhoods.csv")
reviews = pd.read_csv("reviews.csv")
reviews2 = pd.read_csv("reviews2.csv")

For a start, we will be looking at the listings2 datsset. From the list of data fields, we can see that this dataset is reasonably extensive, **with listing details, and reviews related data**. There are also a number of text fields describing the listing that may prove to be useful for our Naive Bayes purposes.

As Airbnb compels each user to write a review after each stay, the **number of reviews is a good proxy of the popularity of the listing**. Should Airbnb be able to predict a popular location in advance, it can give the location more airtime/exposure, and improve the overall experience to users, as users are able to find something they like more quickly. 

In [28]:
list(listings2)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',
 'square_feet',
 'price',
 'weekly_price',


If we look at the end of the list, we see an interesting feature. Reviews per month would weigh the number of reviews against the duration which the location has been listed on Airbnb, giving a more normalized sensing of popularity. 

As starters, **I will use reviews per month as the response variable (y), and summary as the feature (X)**. As the summary field contains text, I can use what we have been learning in class to see if it is predictive. 

In [35]:
listings2[['summary', 'reviews_per_month']].head()

,summary,reviews_per_month
0,This is a big well designed space. Our neighbo...,0.28
1,Beautiful charmng multi- bedroom home in prest...,NaN
2,Convenience is the key for your travel. Walkin...,2.33
3,"When I travel, my quiet beach side condo makes...",NaN
4,Our family-friendly condo is located in San Fr...,0.36


As we can immediately observe that there are some NAs, we will drop NAs from the field and **name the subset summary_to_reviews**.

In [39]:
summary_to_reviews = listings2[['summary', 'reviews_per_month']].dropna()
summary_to_reviews.head()

,summary,reviews_per_month
0,This is a big well designed space. Our neighbo...,0.28
2,Convenience is the key for your travel. Walkin...,2.33
4,Our family-friendly condo is located in San Fr...,0.36
5,Large 2BD apt with back yard and ocean view. F...,1.13
6,Newly built cottage that comfortably sleeps 2....,6.09


At this juncture, I'm wondering if NA means zero/blank, or NA means missing data?. Either way, that's an aspect of the data I might want to keep in mind for later rounds. In the meantime, the subset data, with 5000+ rows, should do as a first attempt.

With summary_to_reviews, we will now attempt to answer the question **"Are we able to predict the popularity of a listing just from its summary description?"**. This can be a question from a marketing or copywriting standpoint.